# Исследование надежности заемщиков


In [ ]:
# импорт библиотеки pandas
import pandas as pd

## Откроем таблицу и изучим общую информацию о данных

In [ ]:
data = pd.read_csv('/datasets/data.csv')

**Выведем первые 20 строчек датафрейма `data` на экран.**

In [ ]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Выведем количество пропущенных значений для каждого столбца.**

In [ ]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` заполним медианным доходом среди всех записей с тем же типом.**

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**Заменим все отрицательные значения столбца `days_employed` положительными с помощью метода `abs()`.**

In [ ]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Оставим их как есть, т.к. этот столбец не понадобится нам для исследования.

**Выведем перечень уникальных значений столбца `children`.**

In [ ]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [ ]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.**

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца.**

In [ ]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведём их к нижнему регистру. Проверим остальные столбцы.**

In [ ]:
data['education'] = data['education'].str.lower()

**Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [ ]:
data.duplicated().sum()

71

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

**На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 назначим категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Создадим функцию `categorize_income()` и применим метод `apply()`.**

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [ ]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` появится строка `'операции с автомобилем'`.**

**Создадим функцию с именем `categorize_purpose()` и применим метод `apply()`.**

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Исследование данных

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [ ]:
# Создадим сводную таблицу, с общим количестом заемщиков и количеством должников в зависимости от количества детей
# Для этого применим к датафрейму группировку по количеству детей, а к значениям из столбца 'debt'
# применим функции 'count' - общее количество заёмщиков и 'sum' - количество должников

children_debt = data.groupby('children').agg({'debt':['count','sum']})

# Добавим столбец 'ratio', где будет видно процентное соотношение количества должников
# к общему количеству заемщиков в зависимости от количества детей у клиентов.

children_debt['ratio'] = (children_debt['debt']['sum'] / children_debt['debt']['count']) * 100
children_debt

debt           ratio
          count   sum          
children                       
0         14091  1063  7.543822
1          4808   444  9.234609
2          2052   194  9.454191
3           330    27  8.181818
4            41     4  9.756098
5             9     0  0.000000

In [ ]:
# Для наглядности выведем на экран результаты, округлив значения до двух символов после запятой

print('Процент клиентов, не вернувших кредит в срок')
print('Клиенты без детей:', round(children_debt['ratio'][0], 2),'%')
print('Клиенты с одним ребёнком:', round(children_debt['ratio'][1], 2),'%')
print('Клиенты с двумя детьми:', round(children_debt['ratio'][2], 2),'%')
print('Клиенты с тремя детьми:', round(children_debt['ratio'][3], 2),'%')
print('Клиенты с четырьмя детьми:', round(children_debt['ratio'][4], 2),'%')

Процент клиентов, не вернувших кредит в срок
Клиенты без детей: 7.54 %
Клиенты с одним ребёнком: 9.23 %
Клиенты с двумя детьми: 9.45 %
Клиенты с тремя детьми: 8.18 %
Клиенты с четырьмя детьми: 9.76 %


<div class="alert-info">  
В целом - появление ребёнка в семье увеличивает вероятность появления задолженности по кредиту.
Но на мой взгляд, данные по клиентам без детей, с одним ребёнком и с двумя детьми - более показательны, т.к. общее количество заёмщиков в этих категориях - достаточно внушительное, в то время, как заёмщиков с тремя детьми всего 330, с четырьмя детьми - 41, с пятью - 9.
Но общая тенденция в любом случае проглядывается.</div>


<div class="alert-info">
<b>Вывод: Наличие ребёнка в семье клиента увеличивает возможность появления задолженности по кредиту, а если сравнить только самые многочисленные выборки с 0, 1 и 2 детьми, можно сказать, что чем больше у заемщика детей, тем выше риск допустить просрочку.</b></div>

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
# Создадим сводную таблицу, с общим количестом заемщиков и количеством должников в зависимости от семейного положения
# Для этого применим к датафрейму группировку по семейному положению, а к значениям из столбца 'debt'
# применим функции 'count' - общее количество заёмщиков и 'sum' - количество должников

family_debt = data.groupby('family_status').agg({'debt':['count','sum']})

# Добавим столбец 'ratio', где будет видно процентное соотношение количества должников
# к общему количеству заемщиков в зависимости от семейного положения клиентов.

family_debt['ratio'] = (family_debt['debt']['sum'] / family_debt['debt']['count']) * 100
family_debt

debt          ratio
                       count  sum          
family_status                              
Не женат / не замужем   2796  273  9.763948
в разводе               1189   84  7.064760
вдовец / вдова           951   63  6.624606
гражданский брак        4134  385  9.313014
женат / замужем        12261  927  7.560558

In [ ]:
# Для наглядности выведем на экран результаты, округлив значения до двух символов после запятой

print('Процент клиентов, не вернувших кредит в срок')
print('Вдовец / вдова:', round(family_debt['ratio'][2], 2),'%')
print('В разводе:', round(family_debt['ratio'][1], 2),'%')
print('Женат / замужем:', round(family_debt['ratio'][4], 2),'%')
print('Гражданский брак:', round(family_debt['ratio'][3], 2),'%')
print('Не женат / не замужем:', round(family_debt['ratio'][0], 2),'%')

Процент клиентов, не вернувших кредит в срок
Вдовец / вдова: 6.62 %
В разводе: 7.06 %
Женат / замужем: 7.56 %
Гражданский брак: 9.31 %
Не женат / не замужем: 9.76 %


<div class="alert-info">   
Самые ответственные плательщики - это вдовы и вдовцы.
    
Если разделить клиентов на тех кто был когда-либо <b>в браке</b> и тех, кто <b>не</b> был <b>в браке</b>, то получается, что официальный статус семьянина положительно сказывается на отношении к выполненнию обязательств по кредиту.</div>

<div class="alert-info">
<b>Вывод:</b>
    
Зависимость между семейным положением и возвратом кредита в срок <b>ЕСТЬ</b>.
Клиенты, не состоящие в браке официально чаще имеют просрочки с выплатами по долгу.</div>

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
# Создадим сводную таблицу, с общим количестом заемщиков и количеством должников в зависимости от уровнем дохода
# Для этого применим к датафрейму группировку по категории дохода, а к значениям из столбца 'debt'
# применим функции 'count' - общее количество заёмщиков и 'sum' - количество должников

income_debt = data.groupby('total_income_category').agg({'debt':['count','sum']})

# Добавим столбец 'ratio', где будет видно процентное соотношение количества должников
# к общему количеству заемщиков в зависимости от дохода клиентов.

income_debt['ratio'] = (income_debt['debt']['sum'] / income_debt['debt']['count']) * 100
income_debt

debt           ratio
                       count   sum          
total_income_category                       
A                         25     2  8.000000
B                       5014   354  7.060231
C                      15921  1353  8.498210
D                        349    21  6.017192
E                         22     2  9.090909

**Категории дохода клиента:**

- 1000001 и выше — `'A'`;
- 200001–1000000 — `'B'`;
- 50001–200000 — `'C'`;
- 30001–50000 — `'D'`;
- 0–30000 — `'E'`.

<div class="alert-info">
На мой взгляд, использование клиентов с доходом категорий "А" и "E" будет не показательно, т.к. количество заёмщиков в этих категориях слишком мало (22 и 25) </div>

In [ ]:
# Для наглядности выведем на экран результаты, округлив значения до двух символов после запятой

print('Процент клиентов, не вернувших кредит в срок')
#print('1000001 и выше:', round(income_debt['ratio'][0], 2),'%')
print('200001–1000000:', round(income_debt['ratio'][1], 2),'%')
print('50001–200000:', round(income_debt['ratio'][2], 2),'%')
print('30001–50000:', round(income_debt['ratio'][3], 2),'%')
#print('0–30000:', round(income_debt['ratio'][4], 2),'%')

Процент клиентов, не вернувших кредит в срок
200001–1000000: 7.06 %
50001–200000: 8.5 %
30001–50000: 6.02 %


<div class="alert-info">   
Самые ответственные плательщики - это клиенты, у которых доход <b>от 30001 до 50000</b>.


<div class="alert-info">
<b>Вывод:</b>

Бóльший доход клиента <b>НЕ</b> означает, что он вовремя выплатит долг по кредиту.
    
Зависимость между уровнем дохода и возвратом кредита в срок конечно есть, но не прямая.
    
В любом случае, полученные данные можно использовать в дальнейшем.
В то же время, если сравнить только две самые большие категории C (50 - 200) и B (200 - 1 млн), выходит что заемщики с более высоким доходом являются более надежными.

Также тут напрашивается рекомендация банку - изменить градацию по доходам, для получения более сбалансированных выборок.</div>

#### Как разные цели кредита влияют на его возврат в срок?

In [ ]:
# Создадим сводную таблицу, с общим количестом заемщиков и количеством должников в зависимости от целей кредита
# Для этого применим к датафрейму группировку по целям кредита, а к значениям из столбца 'debt'
# применим функции 'count' - общее количество заёмщиков и 'sum' - количество должников

purpose_debt = data.groupby('purpose_category').agg({'debt':['count','sum']})

# Добавим столбец 'ratio', где будет видно процентное соотношение количества должников
# к общему количеству заемщиков в зависимости от целей кредита.

purpose_debt['ratio'] = (purpose_debt['debt']['sum'] / purpose_debt['debt']['count']) * 100
purpose_debt

debt          ratio
                          count  sum          
purpose_category                              
операции с автомобилем     4279  400  9.347978
операции с недвижимостью  10751  780  7.255139
получение образования      3988  369  9.252758
проведение свадьбы         2313  183  7.911803

In [ ]:
# Для наглядности выведем на экран результаты, округлив значения до двух символов после запятой

print('Процент клиентов, не вернувших кредит в срок')
print('Операции с недвижимостью:', round(purpose_debt['ratio'][1], 2),'%')
print('Проведение свадьбы:', round(purpose_debt['ratio'][3], 2),'%')
print('Получение образования:', round(purpose_debt['ratio'][2], 2),'%')
print('Операции с автомобилем:', round(purpose_debt['ratio'][0], 2),'%')

Процент клиентов, не вернувших кредит в срок
Операции с недвижимостью: 7.26 %
Проведение свадьбы: 7.91 %
Получение образования: 9.25 %
Операции с автомобилем: 9.35 %


<div class="alert-info">
Кредит, оформленный на операции с недвижимостью возвращается в срок гораздо чаще, чем кредит на операции с автомобилем.</div>

<div class="alert-info"><b>Вывод:</b>
    
Самой востребованной целью кредита являются операции с недвижимостью, и клиенты чаще возвращают заёмные средства в срок.
    
Второй категорией по востребованности являются операции с автомобилем, но количество должников здесь уже заметно выше. Автокредиты выглядят наиболее проблемными.</div>


#### Возможные причины появления пропусков в исходных данных.

Так как пропуски допущены в столбцах "общий трудовой стаж" и "ежемесячный доход", предполагаю, что пропуски допущены в данных из-за особенностей, связанных с работой, либо её отсутствием. Например у пенсионеров такая информация является неактуальной, у госслужащих, возможно, она засекречена, в связи с занимаемой должностью.
Также пропуски могут быть вызваны техническими причинами, например какими то сбоями при выгрузке данных.

### Общий вывод.

<div class="alert-info">
В данном проекте мной были заполнены пропуски в данных, удалены дубликаты, а также проведена категоризация данных - категории клиентов по доходам, а так же категории кредитов по целям кредитования. <br>Далее в сводных таблицах я проанализировал, какие критерии влияют на своевременность возврата кредита клиентами.


Подводя итог, можно сказать, что не все результаты - предсказуемы. Например, увеличение дохода клиента не означает, что кредит будет погашен вовремя.

Но все зависимости своевременной выплаты долга от таких критериев как "количество детей", "семейное положение", "уровень дохода" и "цель кредита" - налицо.

Полученные данные очень показательны, и могут быть использованы в дальнейшем для определения платёжеспособности потенциального заёмщика.

Например самый "ответственный" плательщик, исходя из результатов, - это клиент без детей, но когда-либо состоявший в официальном браке; с доходом от 30000 до 50000 и берущий кредит для операций с недвижимостью.<br>
А вот если заявку на кредит для получения образования либо операций с автомобилем заполнил неженатый/незамужний клиент с доходом от 50000 до 200000, да ещё и с двумя детьми, то стоит более пристально изучить его кредитную историю.</div>